In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time

driver = webdriver.Chrome()

driver.get("https://www.coursera.org/search?query=")

courses_collected = 0
target_courses = 10000 
scroll_pause_time = 0.8  
scroll_increment = 1500  

# Set to store course titles to avoid duplicates
seen_courses = set()
course_data = []

# Infinite scroll and scrape
while courses_collected < target_courses:
    cards = driver.find_elements(By.CSS_SELECTOR, "div.cds-ProductCard-gridCard")

    for card in cards:
        try:
            title = card.find_element(By.CSS_SELECTOR, "h3.cds-CommonCard-title").text.strip()
        except NoSuchElementException:
            title = ""

        if title and title in seen_courses:
            continue

        try:
            partner = card.find_element(By.CSS_SELECTOR, "div.cds-ProductCard-partners p.cds-ProductCard-partnerNames").text.strip()
        except NoSuchElementException:
            partner = ""

        try:
            rating = card.find_element(By.CSS_SELECTOR, "div.cds-RatingStat-sizeLabel span.css-6ecy9b").text.strip()
        except NoSuchElementException:
            rating = ""

        try:
            reviews = card.find_element(By.CSS_SELECTOR, "div.cds-RatingStat-sizeLabel div.css-vac8rf").text.strip()
        except NoSuchElementException:
            reviews = ""

        try:
            description = card.find_element(By.CSS_SELECTOR, "div.cds-CommonCard-metadata p.css-vac8rf").text.strip()
        except NoSuchElementException:
            description = ""

        try:
            course_link = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        except NoSuchElementException:
            course_link = ""

        if title:
            seen_courses.add(title)
            course_data.append({
                "title": title,
                "partner": partner,
                "rating": rating,
                "reviews": reviews,
                "description": description,
                "course_link": course_link
            })
            courses_collected += 1

            print(f"Scraped {courses_collected}/{target_courses} courses.", end="\r")

            if courses_collected >= target_courses:
                break

    # Scroll down for new content
    driver.execute_script(f"window.scrollBy(0, {scroll_increment});")
    time.sleep(scroll_pause_time)

# Close the main driver after collecting course links
driver.quit()


df = pd.DataFrame(course_data)

df.to_csv("coursera_courses.csv", index=False)

print(f"\nCollected {courses_collected} courses.")
for idx, course in enumerate(course_data, start=1):
    print(f"Course {idx}: {course}")


# enrollment 

In [16]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import threading
import queue
import time
import random

class SafePrint:
    """Thread-safe printing"""
    _lock = threading.Lock()

    @classmethod
    def print(cls, *args, **kwargs):
        with cls._lock:
            print(*args, **kwargs)

def scrape_course_enrollment(course_data):
    """
    Scrape enrollment for a single course with improved performance
    
    Args:
        course_data (dict): Dictionary containing course information
    
    Returns:
        dict: Updated course data with enrollment information
    """
    # Headers to mimic browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    try:
        # Send GET request with a timeout
        response = requests.get(
            course_data['course_link'], 
            headers=headers, 
            timeout=10
        )
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Enrollment selectors (most specific to least specific)
        enrollment_selectors = [
            'p.css-4s48ix span strong span',
            'span[data-testid="enrolled-count"]',
            'span.enrollmentNumber',
            'div.enrollment-count'
        ]
        
        # Find enrollment number
        enrollment_element = None
        for selector in enrollment_selectors:
            enrollment_element = soup.select_one(selector)
            if enrollment_element:
                break
        
        if enrollment_element:
            # Clean enrollment number
            enrollment = enrollment_element.text.strip()
            cleaned_enrollment = enrollment.replace(',', '').replace(' already enrolled', '')
            
            # Validate and convert to integer
            try:
                int_enrollment = int(cleaned_enrollment)
                course_data['enrollment'] = int_enrollment
                SafePrint.print(f"Updated {course_data['title']}: {int_enrollment} enrollments")
            except ValueError:
                SafePrint.print(f"Invalid enrollment format for {course_data['title']}: {enrollment}")
                course_data['enrollment'] = None
        else:
            SafePrint.print(f"No enrollment found for {course_data['title']}")
            course_data['enrollment'] = None
        
        return course_data
    
    except requests.RequestException as e:
        SafePrint.print(f"Network error updating {course_data['title']}: {e}")
        course_data['enrollment'] = None
        return course_data
    except Exception as e:
        SafePrint.print(f"Unexpected error with {course_data['title']}: {e}")
        course_data['enrollment'] = None
        return course_data

def update_coursera_enrollments(input_csv, output_csv=None, max_workers=10):
    """
    Update enrollment numbers concurrently
    
    Args:
        input_csv (str): Input CSV file path
        output_csv (str, optional): Output CSV file path
        max_workers (int, optional): Maximum number of concurrent threads
    
    Returns:
        pandas.DataFrame: Updated DataFrame
    """
    # Read input CSV
    try:
        df = pd.read_csv(input_csv)
    except Exception as e:
        print(f"Error reading input CSV: {e}")
        return None

    # Convert DataFrame to list of dictionaries
    courses = df.to_dict('records')

    # Use ThreadPoolExecutor for concurrent processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Map the scraping function to all courses
        updated_courses = list(executor.map(scrape_course_enrollment, courses))

    # Convert back to DataFrame
    updated_df = pd.DataFrame(updated_courses)

    # Save to CSV
    output_path = output_csv or input_csv
    updated_df.to_csv(output_path, index=False)
    print(f"Updated enrollments saved to {output_path}")

    return updated_df

# Main execution
if __name__ == "__main__":
    input_file = 'coursera_courses.csv'
    
    # Measure execution time
    start_time = time.time()
    
    # Update enrollments
    updated_df = update_coursera_enrollments(input_file, max_workers=20)
    
    # Print execution time
    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")
    
    # Optional: print updated dataframe
    if updated_df is not None:
        print("\nUpdated Dataframe:")
        print(updated_df)

Updated Google Prompting Essentials: 12837 enrollments
Updated Google Project Management:: 1806484 enrollments
Updated Google Data Analytics: 2467180 enrollments
Updated Google IT Support: 1695364 enrollments
Updated Microsoft Power BI Data Analyst: 220466 enrollments
Updated IBM Data Science: 656786 enrollments
Updated Google Digital Marketing & E-commerce: 830439 enrollments
Updated Google Cybersecurity: 758422 enrollments
Updated Google AI Essentials: 895109 enrollments
Updated IBM AI Developer: 137446 enrollments
Updated IBM Full Stack Software Developer: 187763 enrollments
Updated IBM Data Analyst: 322279 enrollments
Updated Deep Learning: 889552 enrollments
Updated Google Advanced Data Analytics: 145268 enrollments
Updated Machine Learning: 533022 enrollments
Updated AI For Everyone: 1529946 enrollments
Updated IBM Generative AI Engineering: 2210 enrollments
Updated IBM DevOps and Software Engineering: 82173 enrollments
No enrollment found for Data Science Foundations
Updated AI 

Updated Microsoft Cloud Support Associate: 5371 enrollments
Updated Microsoft IT Support Specialist: 19600 enrollments
Updated IBM Applied DevOps Engineering: 8031 enrollments
Updated Preparing for Google Cloud Certification: Cloud Architect: 119319 enrollments
Updated Start the UX Design Process: Empathize, Define, and Ideate: 540989 enrollments
Updated Business Strategy: 72459 enrollments
Updated Fintech: Foundations & Applications of Financial Technology: 48503 enrollments
Updated Tools of the Trade: Linux and SQL: 256234 enrollments
Updated The Business of Health Care: 22694 enrollments
Updated Process Data from Dirty to Clean: 719682 enrollments
Updated Introduction to Statistics: 521816 enrollments
Updated Agile Project Management: 549860 enrollments
Updated Architecting with Google Compute Engine: 187747 enrollments
Updated Preparing for Google Cloud Certification: Cloud Engineer: 109393 enrollments
Updated Certified in Cybersecurity: 16005 enrollments
Updated Cybersecurity Fund

Updated IT Fundamentals for Cybersecurity: 58400 enrollments
Updated Learning How to Learn: Powerful mental tools to help you master tough subjects: 3916015 enrollments
Updated OKR Certification: Leadership and Goal Setting: 35387 enrollments
Updated Pricing Strategy Optimization: 18223 enrollments
Updated Python 3 Programming: 245416 enrollments
Updated Successful Negotiation: Essential Strategies and Skills: 1600539 enrollments
Updated Introduction to Financial Accounting: 341555 enrollments
Updated Foundations of Data Science: 179862 enrollments
Updated Think Outside the Inbox: Email Marketing: 176399 enrollments
Updated AI Fundamentals for Non-Data Scientists: 35029 enrollments
Updated Convolutional Neural Networks: 524545 enrollments
No enrollment found for Mental Health Care Fundamentals
Updated Modern and Contemporary Art and Design: 62300 enrollments
Updated Career Success: 211317 enrollments
Updated Cybersecurity for Everyone: 405134 enrollments
Updated Medical Terminology: 23

Updated Academic English: Writing: 234048 enrollments
Updated Ciberseguridad de Google: 758422 enrollments
Updated Software Design and Architecture: 80420 enrollments
Updated The Integration of Art and Science in Modern Dental Medicine: 2367 enrollments
Updated Camino a la Excelencia en Gestión de Proyectos: 306931 enrollments
Updated First Step Korean: 1604048 enrollments
Updated Artificial Intelligence: an Overview: 5674 enrollments
Updated Cancer Biology: 27365 enrollments
Updated Managing Emotions in Times of Uncertainty & Stress: 295808 enrollments
Updated Python Data Structures: 1077388 enrollments
Updated Excel Basics for Data Analysis: 389612 enrollments
Updated Software Product Management: 84952 enrollments
Updated Animal Behaviour and Welfare: 157097 enrollments
Updated MLOps | Machine Learning Operations: 15057 enrollments
Updated Microsoft Full-Stack Developer: 2149 enrollments
Updated Accelerated Computer Science Fundamentals: 47117 enrollments
Updated Introduction to Tens

Updated Preparation for CompTIA A+ Certification: 8449 enrollments
Updated Digital Product Management: Modern Fundamentals: 196286 enrollments
Updated Tools for Data Science: 501540 enrollments
Updated AI-Powered Software and System Design: 4122 enrollments
Updated Python Project for Data Science: 239458 enrollments
Updated Learn to Program: The Fundamentals: 496617 enrollments
Updated Music Production: 50360 enrollments
Updated Effective Communication: Writing, Design, and Presentation: 113143 enrollments
Updated Go Beyond the Numbers: Translate Data into Insights: 58173 enrollments
Updated Anatomy: 79012 enrollments
Updated Advanced Valuation and Strategy - M&A, Private Equity, and Venture Capital: 152292 enrollments
Updated Investment Management with Python and Machine Learning: 44057 enrollments
Updated Cybersecurity Operations Fundamentals: 16095 enrollments
Updated Everyday Excel: 71795 enrollments
Updated Introduction to Artificial Intelligence (AI): 451745 enrollments
Updated F

...

Updated Essential Linear Algebra for Data Science: 9061 enrollments
Updated C++ Object Basics: Functions, Recursion, and Objects: 15231 enrollments
No enrollment found for Power System Stability
Updated Natural Language Processing in Microsoft Azure: 8492 enrollments
No enrollment found for Introduction to Archaeology: Knowing The Past
No enrollment found for Load Flow Analysis
Updated Plant-Based Diets: 4750 enrollments
Updated Microservices and Deployment by using ASP.NET: 3202 enrollments
Updated Agile Leadership: Introduction to Change: 45948 enrollments
Updated Computer Vision in Microsoft Azure: 8209 enrollments
No enrollment found for Infraestructura sostenible en la agenda de desarrollo
Updated Exam Prep: AWS Certified DevOps Engineer - Professional: 1655 enrollments
Updated Marketing Channel Functions: 1663 enrollments
Updated Embedded Hardware and Operating Systems: 56695 enrollments
Updated Spring MVC, Spring Boot and Rest Controllers: 25546 enrollments
Updated International

No enrollment found for Introduction to Data Networks and the Internet - Master's
Updated Linux OS - Master's: 3867 enrollments
No enrollment found for Cloud Computing Technologies - Master's
Updated Personnel & Third-Party Security: 2444 enrollments
Updated Marketing estratégico: claves para competir con éxito: 22490 enrollments
No enrollment found for Análisis cualitativo y cuantitativo del riesgo de desastres
No enrollment found for Modelling, Analysis and Design of Steel Buildings
Updated Princípios de Desenvolvimento Ágil de Software: 30757 enrollments
Updated Introduction to iOS App Development with Swift 5: 13495 enrollments
No enrollment found for Riesgos de desastres en proyectos de infraestructura
No enrollment found for Biologia do Sono e Saúde Humana
Updated Introdução à Ciência da Computação com Python Parte 2: 33853 enrollments
No enrollment found for Resource and Waste Management in Buildings
Updated Contexto de Negocios en LATAM: Factores Políticos, Sociales y Económico

No enrollment found for The Theory of the Firm
Updated Data Analysis in Python: Using Pandas DataFrames: 1902 enrollments
No enrollment found for Basics of Fire Protection, Detection and Life Safety Systems
No enrollment found for Public Goods, Common Resources, and Externalities
Updated Detecting COVID-19 with Chest X-Ray using PyTorch: 12529 enrollments
No enrollment found for Data for Effective Policy Making
Updated Linux for Developers: 42098 enrollments
No enrollment found for Electric Vehicle Operation and Diagnosis
Updated Modelos de diseño instruccional para ambientes digitales: 7496 enrollments
Updated Introduction to Bash Shell Scripting: 20324 enrollments
No enrollment found for De-identifying DICOM Data with the Healthcare API
Updated Introduction to Blockchain for Global Commerce: 4597 enrollments
Updated Digital Signal Processing 2: Filtering: 17125 enrollments
Updated ChatGPT for Beginners: SciFi Writing with Dall-e: 17539 enrollments
Updated Create the User Interface wi

No enrollment found for GenAI for Sales Teams
No enrollment found for Scenario and Root Cause Analysis with Generative AI
Updated Cloud Computing Primer: Software as a Service (SaaS): 3186 enrollments
No enrollment found for Gestión musical en América Latina
Updated Primeros Auxilios Psicológicos: 6408 enrollments
Updated Create Informative Presentations with Google Slides: 30591 enrollments
Updated Desarrollo rápido de productos innovadores para mercados emergentes: 51264 enrollments
Updated Software Engineering: Implementation and Testing: 11922 enrollments
No enrollment found for Educación Basada en Evidencias
Updated Responsible AI in the Generative AI Era: 2023 enrollments
Updated Social Pedagogy across Europe: 5365 enrollments
Updated Introduction to IT Security: 2510 enrollments
Updated Algorithmic Thinking (Part 1): 55973 enrollments
Updated Retrieve Data using Single-Table SQL Queries: 8398 enrollments
Updated Introduction to Neurohacking In R: 24087 enrollments
Updated IBM CO

Updated Data Mining Project: 2797 enrollments
Updated Foundations of Information Systems for Business: 2739 enrollments
Updated Control Flow in RPA: 3256 enrollments
Updated Data Mining Pipeline: 8446 enrollments
No enrollment found for Managing Innovation: Ambidexterity, Teams and Leading Change
Updated UX and Interface Design for Embedded Systems: 5474 enrollments
No enrollment found for Challenges of Leading Individuals in the Tech Industry
No enrollment found for Real-World Cloud PM 3 of 3: Security, AWS Well-Architected
No enrollment found for Introduction to Cloud Networking Solutions
Updated Software Architecture Patterns for Big Data: 2846 enrollments
No enrollment found for Value Creation and Building Enduring Relationships
No enrollment found for Fundamentals of Compensation Strategies
Updated Data Processing and Feature Engineering with MATLAB: 15285 enrollments
Updated Lean Software Development: 40536 enrollments
No enrollment found for Juniper Cloud Concepts and Open Sourc

# skills scrap

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import concurrent.futures
import threading
import queue
import time
import random

class SafePrint:
    """Thread-safe printing"""
    _lock = threading.Lock()

    @classmethod
    def print(cls, *args, **kwargs):
        with cls._lock:
            print(*args, **kwargs)

def scrape_course_skills(course_data):
    """
    Scrape skills for a single course with improved performance
    
    Args:
        course_data (dict): Dictionary containing course information
    
    Returns:
        dict: Updated course data with skills information
    """
    # Headers to mimic browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
    }

    try:
        # Send GET request with a timeout
        response = requests.get(
            course_data['course_link'], 
            headers=headers, 
            timeout=15
        )
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Multiple selectors to try finding skills
        skills_selectors = [
            # Original selector
            'ul.css-yk0mzy li.css-0 span.css-o5tswl[aria-hidden="true"]',
            
            # Alternative selectors
            'ul[data-testid="skills-list"] li span.css-o5tswl[aria-hidden="true"]',
            'div[data-testid="skills-section"] span.css-o5tswl[aria-hidden="true"]',
            'div.skills-list span.css-o5tswl[aria-hidden="true"]',
            
            # Fallback: try to find any span with skill-like text
            'span.cds-tag-category'
        ]
        
        skills = []
        for selector in skills_selectors:
            skill_spans = soup.select(selector)
            
            # Extract skills, filtering out unwanted elements
            skills = [
                skill.text.strip() 
                for skill in skill_spans 
                if skill.text.strip() and skill.text.strip() != "View all skills"
            ]
            
            if skills:
                break
        
        if skills:
            course_data['skills'] = ', '.join(skills)
            SafePrint.print(f"✓ {course_data['title']}: {len(skills)} skills found")
        else:
            SafePrint.print(f"✗ {course_data['title']}: No skills found")
            course_data['skills'] = None
        
        return course_data
    
    except requests.RequestException as e:
        SafePrint.print(f"✗ {course_data['title']}: Network error - {e}")
        course_data['skills'] = None
        return course_data
    except Exception as e:
        SafePrint.print(f"✗ {course_data['title']}: Unexpected error - {e}")
        course_data['skills'] = None
        return course_data

def update_coursera_skills(input_csv, output_csv=None, max_workers=10):
    """
    Update skills numbers concurrently
    
    Args:
        input_csv (str): Input CSV file path
        output_csv (str, optional): Output CSV file path
        max_workers (int, optional): Maximum number of concurrent threads
    
    Returns:
        pandas.DataFrame: Updated DataFrame
    """
    # Read input CSV
    try:
        df = pd.read_csv(input_csv)
    except Exception as e:
        print(f"Error reading input CSV: {e}")
        return None

    # Convert DataFrame to list of dictionaries
    courses = df.to_dict('records')

    print(f"Starting skills scraping for {len(courses)} courses...")

    # Use ThreadPoolExecutor for concurrent processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Map the scraping function to all courses
        updated_courses = list(executor.map(scrape_course_skills, courses))

    # Convert back to DataFrame
    updated_df = pd.DataFrame(updated_courses)

    # Save to CSV
    output_path = output_csv or input_csv
    updated_df.to_csv(output_path, index=False)
    
    # Count successful skills extractions
    skills_found = updated_df['skills'].notna().sum()
    skills_total = len(updated_df)
    
    print(f"\nSkills Extraction Summary:")
    print(f"Total Courses: {skills_total}")
    print(f"Courses with Skills: {skills_found}")
    print(f"Success Rate: {skills_found/skills_total*100:.2f}%")
    print(f"Updated skills saved to {output_path}")

    return updated_df

# Main execution
if __name__ == "__main__":
    input_file = 'coursera_courses.csv'
    
    # Measure execution time
    start_time = time.time()
    
    # Update skills
    updated_df = update_coursera_skills(input_file, max_workers=30)
    
    # Print execution time
    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Starting skills scraping for 5488 courses...
✓ Google Data Analytics: 39 skills found
✓ Google Digital Marketing & E-commerce: 35 skills found
✓ Deep Learning: 24 skills found
✓ Microsoft Power BI Data Analyst: 33 skills found
✓ Google UX Design: 34 skills found
✗ AI in Healthcare: No skills found
✓ IBM AI Developer: 47 skills found
✓ Google Advanced Data Analytics: 35 skills found
✗ Google AI Essentials: No skills found
✓ Data Science Foundations: 35 skills found
✓ Google Cybersecurity: 36 skills found
✓ Data Science Fundamentals with Python and SQL: 25 skills found
✗ Foundations of Project Management: No skills found
✓ Meta Front-End Developer: 39 skills found
✓ IBM DevOps and Software Engineering: 75 skills found
✓ Machine Learning: 15 skills found
✓ IBM Full Stack Software Developer: 69 skills found
✓ Introduction to Data Science: 20 skills found
✗ AI For Everyone: No skills found
✓ Google Project Management:: 30 skills found
✓ IBM Generative AI Engineering: 80 skills found
✓ IBM D